In [2]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [3]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
bs=52 # 52 - Jeremey, 20 - default

In [5]:
bptt= 70 #70 - Jeremey, 35 - default

In [6]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [7]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
test_tokens=create_data_lm(df_test['tokens'],bs)
train_tokens.shape, valid_tokens.shape, test_tokens.shape

((52, 396650), (52, 43895), (52, 143259))

In [8]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [9]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [10]:
x.shape,y.shape

((52, 70), (52, 70))

In [11]:
x,y

(array([[   41,    42,    43,    40, ...,     2,  1896,   463,     4],
        [ 1071,    23,    67,    39, ...,     2,    98,    11,    63],
        [   54,    34,     7,   190, ...,     2,    87,    34,  5998],
        [    7,  1213,     8,   453, ...,   406,     4,     2,    46],
        ...,
        [   60,   126,   931,     6, ...,   114,    39,   599,    63],
        [    2, 15094,   132,    65, ...,    56,  1350,   209,   281],
        [11783,   658,   318,     3, ...,     6,    64,    79,  1268],
        [   42,    43,    40,     2, ...,   708,   111,     4,     2]]),
 array([[   42,    43,    40,    13, ...,  1896,   463,     4,     2],
        [   23,    67,    39,    18, ...,    98,    11,    63,    28],
        [   34,     7,   190,  1732, ...,    87,    34,  5998,    47],
        [ 1213,     8,   453,    21, ...,     4,     2,    46,     7],
        ...,
        [  126,   931,     6,   126, ...,    39,   599,    63,   314],
        [15094,   132,    65,   411, ...,  1350, 

In [12]:
x.shape,y.shape

((52, 70), (52, 70))

In [13]:
train_tokens.shape

(52, 396650)

## pre-trained Language Model Weights

In [14]:
PATH_AWD_LSTM='/home/kirana/Documents/phd/data/pre-trained/awd_lstm/'

In [15]:
ls {PATH_AWD_LSTM}

bwd_wt103_enc.h5  fwd_wt103_enc.h5  itos_wt103.pkl  wt103_60002/
bwd_wt103.h5      fwd_wt103.h5      wt103_238642/   wt103_tiny.tgz


In [15]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103.h5', map_location=lambda storage, loc: storage)

In [16]:
wgts

OrderedDict([('0.encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('0.encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                

In [17]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103_enc.h5', map_location=lambda storage, loc: storage)

In [18]:
wgts

OrderedDict([('encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                    

In [19]:
itos2=pickle.load(open(f'{PATH_AWD_LSTM}/itos_wt103.pkl','rb'))
stoi2 = collections.defaultdict(lambda: -1, { v: k for k, v in enumerate(itos2) })

In [20]:
wgts['encoder.weight'].shape

torch.Size([238462, 400])

In [21]:
enc_wgts = wgts['encoder.weight'].numpy() # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [22]:
enc_wgts.shape, len(itos)

((238462, 400), 59972)

In [23]:
n_emb=enc_wgts.shape[1]

In [24]:
new_w = np.zeros((len(itos),n_emb), dtype=np.float32) # shape: (60002, 400)

for i, w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r >= 0 else row_m

In [25]:
len(set(itos).difference(set(itos2)))

8910

## Model Architecture

In [26]:
from torch import nn
import torch
from adaptive import *
from torch.autograd import Variable

In [1]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400 #650
n_layers=2
dropout=0.5
wd=1e-7
dropout_e=0.05
dropout_o=0.5

NameError: name 'itos' is not defined

In [62]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [63]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [64]:
device="cuda:0"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


initializing


In [65]:
new_w.shape

(59972, 400)

In [66]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([59972, 400]), torch.Size([59972, 400]))

In [67]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([59972, 400]), 400, 59972)

### Test if model forward works

In [68]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([59972, 400]), torch.Size([59972, 400]))

In [69]:
x.shape

(52, 70)

In [70]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [71]:

model=model.to(device)

## Learner

In [72]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i=0,0,0
        self.model.gen_hidden()
        for k,i in enumerate(range(0,n_batch,self.bptt)):
            seq_len=min(bptt,n_batch-1-i)
            if 1==0:
                 seq_len=bptt if np.random.random() < 0.95 else bppt//np.randint (2,4)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [73]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [74]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

5667

In [75]:
model.freeze_embedding()

In [78]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [79]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [80]:
model.n_inp

59972

In [81]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 6.915533703804016  0.10734451074711979
Batch:1000 6.641852946281433  0.12191621425654739
Batch:1500 6.483897175153096  0.12988059246477981
Batch:2000 6.365321717500686  0.1354814631412737
Batch:2500 6.267933815574646  0.13960275476537645
Batch:3000 6.184136433124542  0.1430493667271609
Batch:3500 6.109240921974182  0.1459187677372247
Batch:4000 6.0408820444345475  0.1484359970574733
Batch:4500 5.979114531940884  0.15055135121258598
Batch:5000 5.922590770244598  0.1523776455162093
Batch:5500 5.868605335755782  0.15417897919616239
Epoch:0 Loss:5.851854463537548
Batch:500 4.941845922470093  0.20311484259366988
Epoch:0 Loss:5.851854463537548 Accuracy:0.15481419266410965 Loss:4.939919740959788 Accuracy:0.20305440927806653


In [82]:
torch.save(model.state_dict(),f'{PATH}/inter/model_state_dict')


In [85]:
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_state_dict')

In [86]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 5.257263704299927  0.1839615465402603
Batch:1000 5.224924208641053  0.1882744581103325
Batch:1500 5.200545578956604  0.19195879868666332
Batch:2000 5.176978331804276  0.19559272757917642
Batch:2500 5.155534839057922  0.19858385430574418
Batch:3000 5.137897287209829  0.2010177741299073
Batch:3500 5.122337084225246  0.20312002467257637
Batch:4000 5.108523438692093  0.20471511911973359
Batch:4500 5.09719988515642  0.20599842214253214
Batch:5000 5.0879387541770935  0.20706566897332668
Batch:5500 5.078569463729858  0.20808422562209042
Epoch:0 Loss:5.075953854127609
Batch:500 4.6575032281875615  0.24829176956415178
Epoch:0 Loss:5.075953854127609 Accuracy:0.20838425698221003 Loss:4.655906221703099 Accuracy:0.24807408637407294


In [88]:
torch.save(model.state_dict(),f'{PATH}/inter/model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_state_dict')

In [89]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.991079061508179  0.219459901958704
Batch:1000 4.984455912590027  0.21931649541854858
Batch:1500 4.983163781166077  0.2191829789876938
Batch:2000 4.980482181549072  0.21937597350031138
Batch:2500 4.976942756843567  0.21938891309499742
Batch:3000 4.974405508677164  0.21956255780160427
Batch:3500 4.971454155104501  0.21984907014455113
Batch:4000 4.968227214813233  0.22008902303874492
Batch:4500 4.965812192387051  0.22027058592769835
Batch:5000 4.964264732074738  0.22048473732173443
Batch:5500 4.961424910718744  0.2207575045878237
Epoch:0 Loss:4.96080824464528
Batch:500 4.614408593177796  0.25315715420246127
Epoch:0 Loss:4.96080824464528 Accuracy:0.2208191332618676 Loss:4.612693315867983 Accuracy:0.2529345767920097


In [91]:
torch.save(model.state_dict(),f'{PATH}/inter/model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_state_dict')

In [92]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.944511531829834  0.2243934191465378
Batch:1000 4.938592306137085  0.22399836418032645
Batch:1500 4.938548517862956  0.2238305986225605
Batch:2000 4.936670704841614  0.22397047951817511
Batch:2500 4.934162049674987  0.22398221029043197
Batch:3000 4.932666409015655  0.22401997582117716
Batch:3500 4.930681243487767  0.22419930601119994
Batch:4000 4.928575139641762  0.22430763606354595
Batch:4500 4.927115262031555  0.22440654479132757
Batch:5000 4.926383550548554  0.22451248497664927
Batch:5500 4.924282171075994  0.22473068178512834
Epoch:0 Loss:4.923899943021511
Batch:500 4.594277975082398  0.25576869255304335
Epoch:0 Loss:4.923899943021511 Accuracy:0.22475789359284506 Loss:4.592723620565314 Accuracy:0.25557557582570034


In [93]:
torch.save(model.state_dict(),f'{PATH}/inter/model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_state_dict')

In [94]:
model.unfreeze_embedding()

In [99]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [96]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.81522833442688  0.24092034125328063
Batch:1000 4.76445365190506  0.24410193446278572
Batch:1500 4.733384199778239  0.24603481002648672
Batch:2000 4.707725883245468  0.2476096268221736
Batch:2500 4.685522439956665  0.24878891244530676
Batch:3000 4.6671091418266295  0.24973572558661303
Batch:3500 4.650594455991473  0.25070621215019906
Batch:4000 4.635843483567238  0.2515208902172744
Batch:4500 4.623060282283359  0.2522147851718797
Batch:5000 4.611836593151093  0.2528299560725689
Batch:5500 4.600167727643793  0.25355635462836784
Epoch:0 Loss:4.596931660086958
Batch:500 4.209645755290985  0.28460111320018766
Epoch:0 Loss:4.596931660086958 Accuracy:0.25373429276951204 Loss:4.208589770196918 Accuracy:0.2844188846944812


In [100]:
torch.save(model.state_dict(),f'{PATH}/inter/model_awd_lstm_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_awd_lstm_state_dict')

In [101]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.472120983123779  0.2620170433819294
Batch:1000 4.463043106555939  0.2618475376516581
Batch:1500 4.4597108294169105  0.26175495513280234
Batch:2000 4.45430913066864  0.2619619606733322
Batch:2500 4.4492032348632815  0.26203429592251776
Batch:3000 4.444517069975535  0.26218947901328404
Batch:3500 4.439909737859454  0.2624864309430122
Batch:4000 4.435047139525413  0.26274314218759537
Batch:4500 4.430997933387756  0.26297742188307976
Batch:5000 4.427720395565033  0.2631809444904327
Batch:5500 4.423125003988092  0.2635321782908656
Epoch:0 Loss:4.422073341451093
Batch:500 4.111566077709198  0.2909478167295456
Epoch:0 Loss:4.422073341451093 Accuracy:0.2635899615916859 Loss:4.110513316387195 Accuracy:0.29066811846584034


In [105]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.380164099693299  0.26724671345949175
Batch:1000 4.373895459651947  0.2672101752012968
Batch:1500 4.372979234059652  0.2668613656759262
Batch:2000 4.370031606197357  0.2668809170052409
Batch:2500 4.366810906219483  0.26686012033224105
Batch:3000 4.364317613601685  0.26694222663342954
Batch:3500 4.3615351548876085  0.26717136059062824
Batch:4000 4.358501172780991  0.2673732934743166
Batch:4500 4.356046318160163  0.267547080039978
Batch:5000 4.354342210960388  0.267666933465004
Batch:5500 4.351343265360052  0.26793727311762894
Epoch:0 Loss:4.350769360832136
Batch:500 4.055047811985016  0.2948230913877487
Epoch:0 Loss:4.350769360832136 Accuracy:0.26797102496916864 Loss:4.054224991532224 Accuracy:0.2945688229048271


In [109]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 4.329011248588562  0.2707351759970188
Batch:1000 4.323262432098389  0.270489021897316
Batch:1500 4.322941750844319  0.270234992424647
Batch:2000 4.321377471923828  0.2701203404814005
Batch:2500 4.319170068740845  0.2700533074319363
Batch:3000 4.3175302015940344  0.27006383855640886
Batch:3500 4.315568382535662  0.2702028364581721
Batch:4000 4.313407599568367  0.27033002446219323
Batch:4500 4.3117308010525175  0.27041869205236435
Batch:5000 4.310661687660217  0.27050616465210914
Batch:5500 4.3083196582794185  0.2707497610856186
Epoch:0 Loss:4.307920305879555
Batch:500 4.018591405391693  0.2974417732954025
Epoch:0 Loss:4.307920305879555 Accuracy:0.27076235082969624 Loss:4.017623946237031 Accuracy:0.297190981903715
Batch:500 4.295131056785584  0.2729868244230747
Batch:1000 4.289779011726379  0.27264946201443674
Batch:1500 4.290217979431152  0.2723124653895696
Batch:2000 4.288860609769821  0.2723180054798722
Batch:2500 4.287141554069519  0.27221275826096536
Batch:3000 4.286151562

In [111]:
torch.save(model.state_dict(),f'{PATH}/inter/model_awd_lstm_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_awd_lstm_state_dict')

In [112]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 4.270901580810547  0.27449341824650764
Batch:1000 4.266571203708649  0.27415303322672846
Batch:1500 4.267286174456278  0.2738575200239817
Batch:2000 4.2661634786129  0.27380962631106376
Batch:2500 4.26467815246582  0.27370099994540215
Batch:3000 4.263938487370809  0.2736396628965934
Batch:3500 4.2626655612673074  0.27372591351185527
Batch:4000 4.261025905966759  0.2738619615100324
Batch:4500 4.259889873716566  0.2739390830993652
Batch:5000 4.259448423194885  0.2739726484119892
Batch:5500 4.2576001930236815  0.2741780830486254
Epoch:0 Loss:4.257417013819379
Batch:500 3.9693155393600463  0.3012730923295021
Epoch:0 Loss:4.257417013819379 Accuracy:0.2741828942773328 Loss:3.968311595764647 Accuracy:0.3009679072924208
Batch:500 4.252395086288452  0.2762373740375042
Batch:1000 4.248245097637176  0.2756901212930679
Batch:1500 4.248980283419291  0.2754011101226012
Batch:2000 4.247845176815987  0.2752838025987148
Batch:2500 4.246460013103485  0.2752360552430153
Batch:3000 4.24596677247

In [113]:
np.exp(3.9), np.exp(3.94)

(49.40244910553017, 51.418601300526916)

In [115]:
torch.save(model.state_dict(),f'{PATH}/inter/model_awd_lstm_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_awd_lstm_state_dict')

In [116]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 4.237756653785706  0.27697638523578644
Batch:1000 4.233073980331421  0.2767612756341696
Batch:1500 4.2345059900283815  0.2763718063533306
Batch:2000 4.233718021392822  0.27629149506241085
Batch:2500 4.232665308570862  0.2761788027346134
Batch:3000 4.232179588476817  0.27613572584092616
Batch:3500 4.231247132982526  0.27612983888813425
Batch:4000 4.229847648441791  0.27621374786272646
Batch:4500 4.229014091862573  0.2762561776737372
Batch:5000 4.228662105798722  0.2763147369235754
Batch:5500 4.227180867281827  0.2764842275191437
Epoch:0 Loss:4.2269811695050405
Batch:500 3.935943042755127  0.30387693893909457
Epoch:0 Loss:4.2269811695050405 Accuracy:0.2764772423377022 Loss:3.9349633768985144 Accuracy:0.3035317909394345
Batch:500 4.225529035568237  0.2779170450568199
Batch:1000 4.221608436822891  0.27761045134067536
Batch:1500 4.222859638055166  0.2771582536002
Batch:2000 4.222073295593262  0.27706512178480625
Batch:2500 4.220785416412354  0.2769953965008259
Batch:3000 4.2203684

In [117]:
np.exp(3.9), np.exp(3.92)

(49.40244910553017, 50.400444778065484)

In [120]:
torch.save(model.state_dict(),f'{PATH}/inter/model_awd_lstm_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_awd_lstm_state_dict')


In [121]:
torch.save (model,f'{PATH}/inter/model_awd_lstm')


In [122]:
torch.save (optimizer,f'{PATH}/inter/optimizer_awd_lstm')

In [123]:
torch.save (learner,f'{PATH}/inter/learner_awd_lstm')

In [80]:
if 1==0:
    learner.lr_find(1e-4,1e-1,valid_tokens,n_batch)
    learner.plot_lrs(50)
    len(learner.lrs)
    len(learner.losses)

Batch:500 7.944759424209595  0.03532285663112998
Batch:1000 7.877773783683777  0.03720142808556557
Batch:1500 7.802010853131613  0.03968857092410326
Batch:2000 7.722838351964951  0.04160571377538145
Batch:2500 7.658665308570862  0.043098285208642485


## Transfer Learning to predict RNN